In [163]:
import matplotlib.pyplot as plt
import numpy as np
import wave
import sys
import pyaudio
import sympy
from sympy import symbols
from sympy.functions import arg
z = symbols('z')

In [154]:
from sympy.utilities.lambdify import lambdify
def evalHz(Hz,w):
    func = lambdify(z,Hz,'numpy')
    return func(np.exp(1j*w))

In [155]:
def playAudio(filename):
    # Set chunk size of 1024 samples per data frame
    chunk = 1024  

    # Open the sound file 
    wf = wave.open(filename, 'rb')

    # Create an interface to PortAudio
    p = pyaudio.PyAudio()

    # Open a .Stream object to write the WAV file to
    # 'output = True' indicates that the sound will be played rather than recorded
    stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                    channels = wf.getnchannels(),
                    rate = wf.getframerate(),
                    output = True)

    # Read data in chunks
    data = wf.readframes(chunk)

    # Play the sound by writing the audio data to the stream
    while data != '':
        stream.write(data)
        data = wf.readframes(chunk)

    # Close and terminate the stream
    stream.close()
    p.terminate()

In [156]:
def recordAudio():
    save_rep = input('Do you want to save your audio in a new file (could be useful for offline processing? (Y/N)\n')
    if save_rep == 'Y' or save_rep == 'y':
        filename = input('What do you want the audio file name to be? {format: <entername>.wav}\n ')
    else:
        filename = "input.wav"
    secs = int(input("How long do you want the input to be (in seconds)?\n")) 
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = secs
    WAVE_OUTPUT_FILENAME = filename
    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
        

    print("Audio recorded!")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    spf = wave.open(WAVE_OUTPUT_FILENAME, 'r')

    #Extract Raw Audio from Wav File
    signal = spf.readframes(-1)
    signal = np.fromstring(signal, 'Int16')

    return signal

In [167]:
#LCCDE coeffs
def HzLCCDE():
    print('I am assuming that this system is LTI\n')
    a_k = []
    b_m = []
    Nz = 0
    Dz = 0
    ylen = int(input('Enter lenght of y[k] (starting with y[n] and going up to the last non-zero coeff of y[k]): '))
    for i in range(0, ylen):
        p = int(input())
        a_k.append(p) #append the ith coeff of y[k] to the list of coeffs of y[k]
        Dz += p*z**(-1*i) 
    
    xlen = int(input('Enter lenght of x[m] (starting with x[n] and going up to the last non-zero coeff of x[m]): '))
    for i in range(0, xlen):
        p = int(input())
        b_m.append(p) #append the ith coeff of x[m] to thr list of coeffs of x[m]
        Nz += p*z**(-1*i)
        
    Hz = Nz/Dz
    return Hz
    

In [168]:
#poles-zeros of H(z)
def HzPoleZero():
    zeros = []
    poles = []
    Hz = 1
    a = int(input('Enter number of zeros: '))
    for i in range(0, a):
        p = complex(input())
        Hz *= (z-p)
        zeros.append(p) #append the ith zero to the list of zeros

    b = int(input('Enter number of poles: '))
    for j in range(0, b):
        k = complex(input())
        Hz /= (z-k)
        poles.append(k) #append the jth pole to the list of poles
    return Hz


In [169]:
#impulse response
def HzImpulseResponse():
    N = int(input('What is the length of your impulse response h[n]? : '))
    h_n = []
    Hz = 0
    print('Please enter the values of h[t], starting from 0 and going to N-1:\n')
    for i in range(0, N):
        p = int(input())
        h_n.append(p) #append the ith impulse to the list of impulses
        Hz += p*z**(-1*i)
    return Hz
    

In [175]:
#ask for name, to personalize the user experience
user_name = input('Please enter your name:\n')
print('Hey',user_name,'!')
print('Thanks for using my audio filtering platform!')
print("Let's start by designing our filter\n")
#ask the user what type of input they want to use
option = input("Would you like to design your own filter using,\nA. LCCDE coefficients?\nB. poles and zeros of transfer function H(z)?\nC. impulse response h[n]?\n")

Please enter your name:
Shivangi
Hey Shivangi !
Thanks for using my audio filtering platform!
Let's start by designing our filter

Would you like to design your own filter using,
A. LCCDE coefficients?
B. poles and zeros of transfer function H(z)?
C. impulse response h[n]?
C


In [176]:
#main block for constructing the filter
Hz = 1
if option == 'A' or option == 'a':
    Hz = HzLCCDE()
elif option == 'B' or option == 'b':
    Hz = HzPoleZero()
elif option == 'C' or option == 'c':
    Hz = HzImpulseResponse()
else:
    option = input('Enter a valid option (A/B/C)')
print(Hz)

What is the length of your impulse response h[n]? : 5
Please enter the values of h[t], starting from 0 and going to N-1:

1
2
1
2
1
1 + 2/z + z**(-2) + 2/z**3 + z**(-4)
